In [61]:
import csv
import numpy as np

filename = "assignments/assignment1/data/ODI-2018_clean_relevant.csv"

with open(filename,'r') as csvfile:
    # Set the right quote character and delimiter
    csv_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]


In [62]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer

feature_encoder = DictVectorizer()
target_encoder = preprocessing.LabelEncoder()


features = []
targets = []

for row in csv_contents:
    targets.append(row.pop('gender', None))
    features.append(row)


In [63]:
feature_to_index = dict()
index_to_feature = dict()

target_to_index = dict()
index_to_target = dict()

def map_features(features):
    index = 0
    for item in features:
        for k, v in item.items():
            unique = "=".join([k,v])
            if not unique in feature_to_index:
                feature_to_index[unique] = index
                index_to_feature[index] = unique
                index += 1
                
def map_targets(targets):
    index = 0
    for item in targets:
        unique = item
        if not unique in target_to_index:
            target_to_index[unique] = index
            index_to_target[index] = unique
            index += 1
                
def transform_features(features):
    size = len(feature_to_index)
    result = []
    for item in features:
        feature = np.zeros(size)
        for k, v in item.items():
            unique = "=".join([k,v])
            feature[feature_to_index[unique]] = 1
        result.append(feature.tolist())
    return result

def transform_targets(targets):
    result = []
    for item in targets:
        unique = item
        result.append(target_to_index[unique])
    return result

map_features(features)
map_targets(targets)

transformed_features = transform_features(features)
transformed_targets = transform_targets(targets)

In [64]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(transformed_features, transformed_targets)

In [65]:
import graphviz 
#dot_data = tree.export_graphviz(clf, out_file=None) 
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=index_to_feature,
                   class_names=index_to_target)  
graph = graphviz.Source(dot_data) 
graph.render(filename="tree") 

'tree.pdf'